# Notebook 3: Case Triage & Response Assistance

**User Story**: US3 — Case Triage & Response Assistance  
**Persona**: Customer Service Representative (CSR)  

Present a case to the Service Agent for priority recommendation, category,
draft response grounded in Knowledge Base articles, and source citations.
Write-back requires explicit CSR confirmation.

In [ ]:
# Cell 2: Environment + Auth Setup
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)

# Verify required environment variables
required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_OPENAI_DEPLOYMENT",
    "SF_INSTANCE_URL",
    "SF_ACCESS_TOKEN",
]
missing = [v for v in required_vars if not os.environ.get(v)]
if missing:
    raise EnvironmentError(f"Missing required environment variables: {missing}")

print("Environment configured successfully.")
print(f"  Azure AI endpoint: {os.environ['AZURE_AI_PROJECT_ENDPOINT'][:40]}...")
print(f"  Salesforce instance: {os.environ['SF_INSTANCE_URL']}")

In [ ]:
# Cell 3: MCP Connections — salesforce-crm + salesforce-knowledge
import sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import McpToolConnection

# Initialize Azure AI Project client
credential = DefaultAzureCredential()
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
    credential=credential,
)

# MCP connections for both servers
crm_mcp = McpToolConnection(
    server_label="salesforce-crm",
    command=f"{sys.executable} -m mcp_servers.salesforce_crm.server",
    env={
        "SF_INSTANCE_URL": os.environ["SF_INSTANCE_URL"],
        "SF_ACCESS_TOKEN": os.environ["SF_ACCESS_TOKEN"],
        "MCP_TRANSPORT": "stdio",
    },
)

knowledge_mcp = McpToolConnection(
    server_label="salesforce-knowledge",
    command=f"{sys.executable} -m mcp_servers.salesforce_knowledge.server",
    env={
        "SF_INSTANCE_URL": os.environ["SF_INSTANCE_URL"],
        "SF_ACCESS_TOKEN": os.environ["SF_ACCESS_TOKEN"],
        "MCP_TRANSPORT": "stdio",
    },
)

print("MCP connections configured:")
print(f"  salesforce-crm: {crm_mcp.server_label}")
print(f"  salesforce-knowledge: {knowledge_mcp.server_label}")

In [ ]:
# Cell 4: Create Service Agent
from pathlib import Path as _Path

# Load service agent system prompt
system_prompt_path = _Path("../agents/service/system_prompt.md")
system_prompt = system_prompt_path.read_text(encoding="utf-8")

# Create the Service Agent with both MCP tool connections
agent = project_client.agents.create_agent(
    model=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o"),
    name="Service Agent — Case Triage",
    instructions=system_prompt,
    toolset=[crm_mcp, knowledge_mcp],
)

# Create a conversation thread
thread = project_client.agents.threads.create()

print(f"Service Agent created: {agent.id}")
print(f"Thread created: {thread.id}")
print(f"Tools: salesforce-crm (cases, accounts), salesforce-knowledge (articles)")

In [ ]:
# Cell 5: Triage request — "Triage case #12345 and suggest a response"

triage_message = (
    "Triage case #12345 and suggest a response. "
    "Include priority recommendation, category, "
    "a draft response grounded in Knowledge Base articles, "
    "and cite the KB article sources."
)

# Send the message
project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=triage_message,
)

# Run the agent
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

print(f"Run status: {run.status}")

In [ ]:
# Cell 6: Display triage results — priority, category, draft response, KB citations
from IPython.display import Markdown, display

messages = project_client.agents.messages.list(thread_id=thread.id)

# Find the most recent assistant message
for msg in messages:
    if msg.role == "assistant":
        for content_block in msg.content:
            if hasattr(content_block, "text"):
                display(Markdown(content_block.text.value))
        break

In [ ]:
# Cell 7: User confirms write-back — "Apply this triage"

confirm_message = (
    "Apply this triage. Update the case priority and category as recommended, "
    "and add the draft response as an internal comment."
)

project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=confirm_message,
)

# Run the agent — it should execute the write-back with confirmed=true
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

# Display the confirmation result
messages = project_client.agents.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant":
        for content_block in msg.content:
            if hasattr(content_block, "text"):
                display(Markdown(content_block.text.value))
        break

In [ ]:
# Cell 8: Verify update in Salesforce

verify_message = "Show me the current details of case #12345 to confirm the update was applied."

project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=verify_message,
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
)

messages = project_client.agents.messages.list(thread_id=thread.id)
for msg in messages:
    if msg.role == "assistant":
        for content_block in msg.content:
            if hasattr(content_block, "text"):
                display(Markdown(content_block.text.value))
        break

In [ ]:
# Cell 9: Cleanup

# Delete the agent and thread
project_client.agents.delete_agent(agent.id)
print(f"Agent {agent.id} deleted.")

project_client.agents.threads.delete(thread.id)
print(f"Thread {thread.id} deleted.")

print("\nCase triage session complete.")